Traditionally Python programmers use [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/) to scrape content from the interent. Instead of being *traditional*, we're going to use [Playwright](https://playwright.dev/python/), a **browser automation tool**! This means you actually control the browser! Filling out forms, clicking buttons, downloading documents... it's magic!!!✨✨✨

# Chicago Building Records

- Clicking
- Filling out forms with `type` instead of `fill`
- Extracting a single table
- Looping through addresses
- CSS selectors?
- Clicking links
- Dataframe manipulation
  Combining dataframes

## Installation

We need to install a few tools first! Remove the `#` and run the cell to install the Python packages and browsers that we'll need for our scraping adventure.

In [1]:
# %pip install --quiet lxml html5lib beautifulsoup4 pandas
# %pip install --quiet playwright
# !playwright install

## Opening up the browser and visiting our destination


In [2]:
from playwright.async_api import async_playwright

# "Hey, open up a browser"
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
context = await browser.new_context()

# Create a new browser window
page = await context.new_page()

In [3]:
await page.goto("https://webapps1.chicago.gov/buildingrecords/home")

<Response url='https://webapps1.chicago.gov/buildingrecords/home' request=<Request url='https://webapps1.chicago.gov/buildingrecords/home' method='GET'>>

## Clicking a button

In [4]:
await page.locator("#rbnAgreement1").click()
await page.get_by_text("Submit").click()

## Filling in a field that *demands* keyboard input

You usually use `.fill` to write in a text box. But some forms want to know someone typed in it! In this case, you'll use `.type` instead.

In [5]:
# await page.locator("input").fill("400 e 41st st")
# await page.get_by_label("Building Address").fill("400 e 41st st")
await page.get_by_label("Building Address").type("400 e 41st st")

In [6]:
await page.get_by_text("Submit").click()

## Grab the data from the page

[Pandas](https://pandas.pydata.org/) is the Python equivalent to Excel, and it's great at dealing with tabular data! Often the data on a web page that looks like a spreadsheet can be read with `pd.read_html`.

In this case, there *isn't one*. You need to use BeautifulSoup to scrape the page manually! *But* you first needed to use Playwright to open hte page, execute the search, and scrollllll to fill up the page first.

In [10]:
import pandas as pd
from io import StringIO

html = await page.content()
tables = pd.read_html(StringIO(html))
len(tables)

4

In [11]:
df = tables[2]
df.head()

,INSP #,INSPECTION DATE,STATUS,TYPE DESCRIPTION
0,14382903,05/30/2024,FAILED,COMPLAINT INSPECTION
1,14258795,05/10/2024,PASSED,ANNUAL INSPECTION
2,14330129,05/01/2024,FAILED,CONSERVATION COMPLAINT INSPECT
3,14214556,03/05/2024,FAILED,ANNUAL INSPECTION
4,14218826,02/23/2024,FAILED,ANNUAL INSPECTION


In [41]:
df.to_csv("output.csv", index=False)

## Getting details of each of the inspections

How many links are on the page? I found `"#resultstable_inspections a"` by knowing how CSS selectors work. It means "links inside of an element with an id of `resultstable_inspections`.

In [14]:
links = page.locator("#resultstable_inspections a")
count = await links.count()
count

179

### Clicking a single link for details

When we click the link it opens up a new page. Below we click one of the links and wait until the "Print" text shows up on the page. We can talk about the new page with `new_page`, while `page` is still the original page.

In [43]:
async with context.expect_page() as new_page_info:
    await links.nth(1).click()

new_page = await new_page_info.value
await new_page.get_by_text("Print").wait_for()

Now we'll pull the content from the new page just like we normally do.

In [2]:
html = await new_page.content()
tables = pd.read_html(StringIO(html))
len(tables)

NameError: name 'new_page' is not defined

It's kind of a weird table, so we need to clean it up a bit. I know I (probably) promised that all of this would be 100% cut-and-paste reuseable but SADLY this time it is not.

In [48]:
df = tables[0]
df.columns = df.columns.droplevel()
df

,VIOLATIONS,BUILDING CODE CITATION,VIOLATION DETAILS
0,FP1295,"SECTION 13-76-030, 13-76-040, 13-76-050, 13-76...",PROVIDE FOR A REQUIRED ANNUAL TEST OF YOUR FIR...


## Putting it all together

Let's experiment with looking through the different links, and then make it work.

In [44]:
links = page.locator("#resultstable_inspections a")
count = await links.count()
count

179

In [15]:
for i in range(3):
    link = links.nth(i)
    inspection_num = await links.nth(i).inner_text()
    print("Inspection number", inspection_num)

Inspection number 14382903
Inspection number 14258795
Inspection number 14330129


In [16]:
for i in range(3):
    link = links.nth(i)
    inspection_num = await links.nth(i).inner_text()
    print("Inspection number", inspection_num)

    async with context.expect_page() as new_page_info:
        await link.click()

    new_page = await new_page_info.value
    await new_page.get_by_text("Print").wait_for()

    await new_page.close()

Inspection number 14382903
Inspection number 14258795
Inspection number 14330129


Okay let's go!!!

In [49]:
all_data = pd.DataFrame()

# for i in range(count):
for i in range(5):
    # Get the link and the link details
    link = links.nth(i)
    inspection_num = await links.nth(i).inner_text()

    # Open the page
    async with context.expect_page() as new_page_info:
        await link.click()

    # Access the page
    new_page = await new_page_info.value
    await new_page.get_by_text("Print").wait_for()

    # Grab the table
    try:
        print("Saving violations for", inspection_num)
        html = await new_page.content()
        tables = pd.read_html(StringIO(html), header=None)
        df = tables[0]
        df.columns = df.columns.droplevel()
        df['inspection_num'] = inspection_num
        all_data = pd.concat([all_data, df], ignore_index = True)
    except:
        print("No violations for", inspection_num)
        
    # Close the page
    await new_page.close()

Saving violations for 14382903
Saving violations for 14258795
No violations for 14258795
Saving violations for 14330129
Saving violations for 14214556
Saving violations for 14218826


In [50]:
all_data

,VIOLATIONS,BUILDING CODE CITATION,VIOLATION DETAILS,inspection_num
0,EV1110,Failed to maintain electric elevator equipment...,Perform Fire Services tests on all Elevators. ...,14382903
1,EV1110,Failed to maintain electric elevator equipment...,Perform the CAT1 test- scheduled with All Types,14382903
2,EV1110,Failed to maintain electric elevator equipment...,Determine reason for the loud noise - from #2 ...,14382903
3,CN031023,Failed to maintain properly charged fire extin...,FIRE EXTINGUISHERS IN ELECTRICAL SERVICE ROOM ...,14330129
4,CN043013,Failed to enclose pipe shafts and ducts passin...,MUST SEAL ALL AND ANY PIPE PENETRATONS IN BOIL...,14330129
5,CN061014,Failed to maintain the exterior walls of a bui...,FRONT ENTRYWAY SPALLING CONCRETE WITH EXPOSED ...,14330129
6,CN065014,Failed to maintain lintel in good repair and f...,LINTELS AT FRONT ENTRYWAY RUSTED,14330129
7,CN140016,"Keep premises clean, sanitary, and safe. (13-1...",ALL MECHANICAL ROOMS SHALL BE CLEAR AND FREE O...,14330129
8,CN197079,Repair or replace defective or out of service ...,SMOKE DETECTOR BEEPING AT 1ST FLOOR BOILER ROO...,14330129
9,EV1110,Failed to maintain electric elevator equipment...,Perform CAT1 TEST and provide documentation,14214556


## Saving the results

Now we'll save it to a CSV file! Easy peasy.

In [51]:
all_data.to_csv("output.csv", index=False)